# 허깅페이스 로그인 인증

In [ ]:
/

In [ ]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

# 1️⃣ .env 파일의 내용을 로드
load_dotenv(override=True)

# 2️⃣ 환경 변수 가져오기
HF_READ_TOKEN = os.getenv("HF_READ_TOKEN")
# print(HF_READ_TOKEN)

# 로그인 실행
login(HF_READ_TOKEN)

## Huggingface transformers 라이브러리 활용하기

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("text-generation")

In [ ]:
pipe("인공지능이란 무엇인가?")

=> text-generation 기본은 gpt-2를 다운 받음  
=> gpt-2는 한국어를 잘 인식하지 못함

In [ ]:
# 영어 인식하기
result = pipe("What is artificial intelligence?")

print(result[0]["generated_text"])

## 한국어 인식이 잘 되는 모델

In [ ]:
# task : text-generation
# model : skt/kogpt2-base-v2
txt_gen_ko = pipeline("text-generation", model="skt/kogpt2-base-v2")


In [ ]:
txt_gen_ko("인공지능에 대해 설명해줘")

In [ ]:
# - max_new_tokens: 256
# - do_sample: True
# - temperature: 0.7 (0.0은 허용하지 않음)
result = txt_gen_ko("토끼는", do_sample = True, temperature=0.7, max_new_tokens = 256)

In [ ]:
result[0]["generated_text"]

In [ ]:
print(result[0]["generated_text"])

In [ ]:
# LGAI-EXAONE/EXAONE-4.0-1.2B

txt_gen_ko = pipeline("text-generation", model="LGAI-EXAONE/EXAONE-4.0-1.2B")


In [ ]:
result = txt_gen_ko("토끼는")
print(result[0]["generated_text"])

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "LGAI-EXAONE/EXAONE-4.0-1.2B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="bfloat16",
    device_map="auto"  # cuda, cpu
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# choose your prompt
prompt = "Explain how wonderful you are"
prompt = "Explica lo increíble que eres"
prompt = "너가 얼마나 대단한지 설명해 봐"

messages = [
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

output = model.generate(
    input_ids.to(model.device),
    max_new_tokens=128,
    do_sample=False,
)
print(tokenizer.decode(output[0]))

## 감성 분석 모델

In [ ]:
classifier = pipeline("text-classification")

In [ ]:
result = classifier("I love using Huggin Face transformers")
result

In [ ]:
# 여러 분장을 한꺼번에 분류처리 할 때
data = ['This restaurant serves delicious food.', "Idon't like to wake up early."]

In [ ]:
result = classifier(data)
result

In [ ]:
# [미션] -  아래와 같이 출력 되도록
# "I love using Huggin Face transformers" - POSITIVE
# "Idon't like to wake up early." - NEGATIVE

In [ ]:
datas = ["I love using Huggin Face transformers", "Idon't like to wake up early."]
result = classifier(datas)
result

# 한국어 감정분석
- 텍스트에서 긍정 또는 부정으로 분류

In [ ]:
data = ['햄버거가 맛이 별로다', '나는 수영을 잘하지 못한다.', ' 이 카페의 커피맛이 예술이네.']
result = classifier(data)
result

- 한국어를 못 분류하기 떄문에 한국어 분류 가능한 모델을 사용해야함.

In [ ]:
from transformers import pipeline
Sentiment_model = pipeline("sentiment-analysis", model="WhitePeak/bert-base-cased-Korean-sentiment")

In [ ]:
data = ['햄버거가 맛이 별로다', '나는 수영을 잘하지 못한다.', ' 이 카페의 커피맛이 예술이네.']
result = Sentiment_model(data)
result

In [ ]:
# label_0 : 부정
# label_1 : 긍정

In [ ]:
# 문제 : 다음과 같이 출력되도록 하기
# 햄버거가 맛이 별로다 -negative
# 나는 수영을 잘하지 못한다. -negative
# 이 카페의 커피맛이 예술이네. -positive

for sen, cl in zip(data, result):
    if cl['label'] == 'LABEL_1':
        label = 'positive'
    else:
        label = 'negative'
    print(f"{sen.strip()} - {label}")

## 한국어 인식이 잘되는 모델

In [ ]:
# google/gemma-3-270m-it, 승인 필요
model="google/gemma-3-270m-it"
# generation = pipeline("text-generation", model=model)
generation = pipeline(
    "text-generation",
    model="google/gemma-3-270m-it",
    tokenizer="google/gemma-3-270m-it",
    token=HF_READ_TOKEN,            # transformers 최신 버전에서 권장
    trust_remote_code=True,    # 필요 시
    device_map="auto"
)